In [11]:
import numpy as np
import pandas as pd
import matplotlib as mplt
import seaborn as sns

import re
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from collections import Counter
import warnings

In [12]:
df_catalog = pd.read_csv("b_product_catalog.csv")
df_descriptions = pd.read_csv("b_unstructured_descriptions.csv")

In [13]:
df_catalog

,SKU,Product_Name,Category,Subcategory,Brand,Color,Size,Material,Features,Season,Price
0,SKU1000000,Nordic Jacket,Outerwear,Jacket,Nordic,Brown,XL,Polyester,Stretch|Breathable,Fall 2025,242.05
1,SKU1000001,Elite Coat,Outerwear,Coat,Elite,Tan,M,Linen,Moisture-wicking,Winter 2024,247.58
2,SKU1000002,Premium Cardigan,Outerwear,Cardigan,Premium,Black,M,Fleece,Quick-dry,Spring 2025,99.55
3,SKU1000003,Alpine Blazer,Outerwear,Blazer,Alpine,Olive,M,Nylon,UV-protection|Waterproof|Quick-dry,Winter 2025,177.36
4,SKU1000004,Alpine Parka,Outerwear,Parka,Alpine,Charcoal,XL,Wool,Windproof|Moisture-wicking|Quick-dry,Summer 2024,215.47
...,...,...,...,...,...,...,...,...,...,...,...
795,SKU1000795,Style Boots,Footwear,Boots,Style,Black,XXL,Fleece,NaN,Spring 2024,186.62
796,SKU1000796,Elite Sneakers,Footwear,Sneakers,Elite,Olive,XL,Blend,NaN,Winter 2024,125.52
797,SKU1000797,Essential Sandals,Footwear,Sandals,Essential,Green,L,Blend,Quick-dry|Breathable,Summer 2025,169.93
798,SKU1000798,Elite Loafers,Footwear,Loafers,Elite,Olive,XXL,Wool,Insulated|Breathable,Summer 2024,205.92


In [14]:
df_descriptions

,Description_ID,Unstructured_Description,Source_Channel
0,DESC0001,Do you carry Style Pants for Fall 2025?,Marketplace
1,DESC0002,"Looking for Green Boots, size L.",Email
2,DESC0003,Searching for the Modern blazer finished in da...,Website
3,DESC0004,Need blue sundress that is breathable and flex...,Website
4,DESC0005,"Looking for Beige Gown, size L.",Marketplace
...,...,...,...
245,DESC0246,Need Classic Blouse Cream size S,Phone
246,DESC0247,"Charcoal Sandals from Modern, size M",Website
247,DESC0248,Do you stock Alpine's t-shirt in that charcoal...,Website
248,DESC0249,Looking for Elite Sundress Red L,Website


In [15]:
print("DATASET A - UNSTRUCTURED DESCRIPTIONS")
print(f"Shape: {df_descriptions.shape}")
print(f"\nColumns: {df_descriptions.columns.tolist()}")
print(f"\nSource channels: {df_descriptions['Source_Channel'].value_counts()}")

print("\n DATASET B - PRODUCT CATALOG")
print(f"Shape: {df_catalog.shape}")
print(f"\nColumns: {df_catalog.columns.tolist()}")


DATASET A - UNSTRUCTURED DESCRIPTIONS
Shape: (250, 3)

Columns: ['Description_ID', 'Unstructured_Description', 'Source_Channel']

Source channels: Source_Channel
Website        60
Email          50
Chat           48
Marketplace    47
Phone          45
Name: count, dtype: int64

 DATASET B - PRODUCT CATALOG
Shape: (800, 11)

Columns: ['SKU', 'Product_Name', 'Category', 'Subcategory', 'Brand', 'Color', 'Size', 'Material', 'Features', 'Season', 'Price']


In [16]:
# The next check is to look for missing values in our catalog data table
print(f"\nMissing values in Catalog:")
print(df_catalog.isnull().sum())


Missing values in Catalog:
SKU               0
Product_Name      0
Category          0
Subcategory       0
Brand             0
Color             0
Size              0
Material          0
Features        169
Season            0
Price             0
dtype: int64


* So there are 169 NULL values in our dataset, specifically in the Features column

In [17]:
# Understand the data and the vocabulary for the catalog data
print(f"\nUnique Brands: {df_catalog['Brand'].unique()}")
print(f"\nUnique Colors: {df_catalog['Color'].unique()}")
print(f"\nUnique Sizes: {df_catalog['Size'].unique()}")
print(f"\nUnique Categories: {df_catalog['Category'].unique()}")
print(f"\nUnique Subcategories: {df_catalog['Subcategory'].unique()}")
print(f"\nSeasons: {df_catalog['Season'].unique()}")


Unique Brands: ['Nordic' 'Elite' 'Premium' 'Alpine' 'Urban' 'Essential' 'Style' 'Modern'
 'Classic' 'Comfort']

Unique Colors: ['Brown' 'Tan' 'Black' 'Olive' 'Charcoal' 'Gray' 'Cream' 'Blue' 'Burgundy'
 'Beige' 'Navy' 'Green' 'White' 'Red']

Unique Sizes: ['XL' 'M' 'L' 'XXL' 'XS' 'S']

Unique Categories: ['Outerwear' 'Tops' 'Bottoms' 'Dresses' 'Footwear']

Unique Subcategories: ['Jacket' 'Coat' 'Cardigan' 'Blazer' 'Parka' 'Vest' 'Shirt' 'Blouse'
 'Sweater' 'T-Shirt' 'Polo' 'Turtleneck' 'Pants' 'Jeans' 'Skirt' 'Shorts'
 'Leggings' 'Dress' 'Gown' 'Sundress' 'Boots' 'Sneakers' 'Sandals'
 'Loafers' 'Heels']

Seasons: ['Fall 2025' 'Winter 2024' 'Spring 2025' 'Winter 2025' 'Summer 2024'
 'Spring 2024' 'Summer 2025' 'Fall 2024']


<b>Clean and preprocess the catalog dataset </b>

In [18]:
# Create a clean copy
catalog_clean = df_catalog.copy()

# Handle missing values in Features
catalog_clean['Features'] = catalog_clean['Features'].fillna('')

In [19]:
# Create a searchable text field combining all relevant columns
def create_searchable_text(row):
    # Combine all relevant fields into searchable text
    parts = [
        str(row['Product_Name']),
        str(row['Brand']),
        str(row['Subcategory']),
        str(row['Color']),
        str(row['Size']),
        str(row['Material']),
        str(row['Features']).replace('|', ' '),
        str(row['Season'])
    ]
    return ' '.join(parts).lower()

catalog_clean['searchable_text'] = catalog_clean.apply(create_searchable_text, axis=1)

# Standardize colors, sizes, etc.
catalog_clean['color_clean'] = catalog_clean['Color'].str.lower().str.strip()
catalog_clean['size_clean'] = catalog_clean['Size'].str.upper().str.strip()
catalog_clean['brand_clean'] = catalog_clean['Brand'].str.lower().str.strip()
catalog_clean['subcategory_clean'] = catalog_clean['Subcategory'].str.lower().str.strip()

print("Sample searchable text:")
print(catalog_clean[['SKU', 'searchable_text']].head(3))

Sample searchable text:
          SKU                                    searchable_text
0  SKU1000000  nordic jacket nordic jacket brown xl polyester...
1  SKU1000001  elite coat elite coat tan m linen moisture-wic...
2  SKU1000002  premium cardigan premium cardigan black m flee...


<b> The next step is to build synonym dictinaries that can help us with the search and matches</b>

In [22]:
catalog_clean['color_clean'].unique() # Gathering all the distinct colors from the catalog

array(['brown', 'tan', 'black', 'olive', 'charcoal', 'gray', 'cream',
       'blue', 'burgundy', 'beige', 'navy', 'green', 'white', 'red'],
      dtype=object)

In [23]:
# Color synonyms
color_synonyms = {
    'navy': 'navy blue',
    'dark blue': 'navy',
    'midnight blue': 'navy',
    'deep navy': 'navy',
    'naavy': 'navy',  # Common misspelling
    'biege': 'beige',
    'beege': 'beige',
    'grey': 'gray',
    'charcoal': 'gray',
    'dark grey': 'gray',
    'burundy': 'burgundy',
    'burguundy': 'burgundy',
    'maroon': 'burgundy',
    'ecru': 'cream',
    'olive tone': 'olive'
}

# Size standardization
size_patterns = {
    r'\bextra small\b': 'XS',
    r'\bx-small\b': 'XS',
    r'\bsmall\b': 'S',
    r'\bmedium\b': 'M',
    r'\blarge\b': 'L',
    r'\bx-large\b': 'XL',
    r'\bextra large\b': 'XL',
    r'\bxx-large\b': 'XXL',
    r'\bextra extra large\b': 'XXL'
}

# Common misspellings in the data
misspelling_corrections = {
    'santals': 'sandals',
    'essetnial': 'essential',
    'boluse': 'blouse',
    'norddic': 'nordic',
    'turleneck': 'turtleneck',
    'clssic': 'classic',
    'patns': 'pants',
    'chaarcoal': 'charcoal',
    'parrka': 'parka',
    'elife': 'elite'
}

In [24]:
def normalize_color(color_text):
    # Normalize color mentions with synonyms
    color_lower = color_text.lower()
    for synonym, standard in color_synonyms.items():
        color_lower = color_lower.replace(synonym, standard)
    return color_lower


In [25]:
def normalize_size(text):
    # Extract and normalize size from text
    text_upper = text.upper()
    # Direct size mentions
    sizes = ['XXL', 'XL', 'L', 'M', 'S', 'XS']
    for size in sizes:
        if re.search(r'\b' + size + r'\b', text_upper):
            return size
    
    # Pattern matching for written sizes
    text_lower = text.lower()
    for pattern, size in size_patterns.items():
        if re.search(pattern, text_lower):
            return size
    return None

In [26]:
def correct_misspellings(text):
    # Fix common misspellings
    text_lower = text.lower()
    for wrong, correct in misspelling_corrections.items():
        text_lower = re.sub(r'\b' + wrong + r'\b', correct, text_lower)
    return text_lower

<b> Preprocess unstructured descriptions dataset </b>

In [28]:
# Create clean descriptions
descriptions_clean = df_descriptions.copy()

def clean_description(text):
    #Clean and normalize description text    
    text = str(text).lower() # Convert to lowercase       
    text = correct_misspellings(text) # Fix misspellings       
    text = normalize_color(text) # Normalize colors       
    text = ' '.join(text.split()) # Remove extra whitespace    
    return text

descriptions_clean['description_clean'] = descriptions_clean['Unstructured_Description'].apply(clean_description)

print("Sample cleaned descriptions:")
print(descriptions_clean[['Description_ID', 'Unstructured_Description', 'description_clean']].head())


Sample cleaned descriptions:
  Description_ID                           Unstructured_Description  \
0       DESC0001            Do you carry Style Pants for Fall 2025?   
1       DESC0002                   Looking for Green Boots, size L.   
2       DESC0003  Searching for the Modern blazer finished in da...   
3       DESC0004  Need blue sundress that is breathable and flex...   
4       DESC0005                    Looking for Beige Gown, size L.   

                                   description_clean  
0            do you carry style pants for fall 2025?  
1                   looking for green boots, size l.  
2  searching for the modern blazer finished in da...  
3  need blue sundress that is breathable and flex...  
4                    looking for beige gown, size l.  


<b> </b>

<b> The next step is to extract arrtibutes from descriptions </b>

In [29]:
def extract_brand(text, brands):
    # Extract brand from text
    text_lower = text.lower()
    for brand in brands:
        if re.search(r'\b' + brand.lower() + r'\b', text_lower):
            return brand
    return None

In [30]:
def extract_color(text, colors):
    # Extract color from text
    text_lower = normalize_color(text.lower())
    for color in colors:
        color_lower = color.lower()
        if re.search(r'\b' + color_lower + r'\b', text_lower):
            return color
    return None

In [31]:
def extract_subcategory(text, subcategories):
    # Extract subcategory/product type from text
    text_lower = text.lower()
    for subcat in subcategories:
        if re.search(r'\b' + subcat.lower() + r'\b', text_lower):
            return subcat
    return None

In [32]:
# Extract season from text
def extract_season(text):    
    # Pattern: Fall 2025, Winter 2024, etc.
    season_pattern = r'(Spring|Summer|Fall|Winter)\s*(\d{4})'
    match = re.search(season_pattern, text, re.IGNORECASE)
    if match:
        return f"{match.group(1).capitalize()} {match.group(2)}"
    return None

In [33]:
# Extract all possible attributes from description
def extract_all_attributes(text, catalog):    
    brands = catalog['Brand'].unique().tolist()
    colors = catalog['Color'].unique().tolist()
    subcategories = catalog['Subcategory'].unique().tolist()
    
    attributes = {
        'brand': extract_brand(text, brands),
        'color': extract_color(text, colors),
        'size': normalize_size(text),
        'subcategory': extract_subcategory(text, subcategories),
        'season': extract_season(text)
    }
    
    return attributes

In [34]:
# Test attribute extraction
test_desc = "Looking for naavy Santals by lite, size L please."
test_attrs = extract_all_attributes(test_desc, catalog_clean)
print(f"\nTest extraction on: '{test_desc}'")
print(f"Extracted attributes: {test_attrs}")

# Apply to all descriptions
descriptions_clean['extracted_attrs'] = descriptions_clean['description_clean'].apply(
    lambda x: extract_all_attributes(x, catalog_clean)
)

print("\nSample extracted attributes:")
for idx in range(3):
    print(f"\n{descriptions_clean.iloc[idx]['Unstructured_Description']}")
    print(f"Extracted: {descriptions_clean.iloc[idx]['extracted_attrs']}")


Test extraction on: 'Looking for naavy Santals by lite, size L please.'
Extracted attributes: {'brand': None, 'color': 'Navy', 'size': 'L', 'subcategory': None, 'season': None}

Sample extracted attributes:

Do you carry Style Pants for Fall 2025?
Extracted: {'brand': 'Style', 'color': None, 'size': None, 'subcategory': 'Pants', 'season': 'Fall 2025'}

Looking for Green Boots, size L.
Extracted: {'brand': None, 'color': 'Green', 'size': 'L', 'subcategory': 'Boots', 'season': None}

Searching for the Modern blazer finished in dark grey, XXL size.
Extracted: {'brand': 'Modern', 'color': 'Gray', 'size': 'XXL', 'subcategory': 'Blazer', 'season': None}


<b> The next step is to build a matching algorithm with scoring so we can determine how good or bad the match works. </b>

In [35]:
# Create TF-IDF vectorizer
tfidf = TfidfVectorizer(
    max_features=500,
    ngram_range=(1, 2),
    stop_words='english',
    min_df=1
)

# Fit on catalog searchable text
catalog_vectors = tfidf.fit_transform(catalog_clean['searchable_text'])
print(f"TF-IDF vectors created: {catalog_vectors.shape}")

TF-IDF vectors created: (800, 500)


In [36]:
#Calculate attribute match score (0-1)
def calculate_attribute_score(query_attrs, catalog_row):    
    score = 0
    max_score = 0    
    # Brand match (weight: 25%)
    if query_attrs['brand']:
        max_score += 0.25
        if catalog_row['brand_clean'] == query_attrs['brand'].lower():
            score += 0.25
    
    # Color match (weight: 25%)
    if query_attrs['color']:
        max_score += 0.25
        if catalog_row['color_clean'] == query_attrs['color'].lower():
            score += 0.25
    
    # Size match (weight: 25%)
    if query_attrs['size']:
        max_score += 0.25
        if catalog_row['size_clean'] == query_attrs['size']:
            score += 0.25
    
    # Subcategory match (weight: 15%)
    if query_attrs['subcategory']:
        max_score += 0.15
        if catalog_row['subcategory_clean'] == query_attrs['subcategory'].lower():
            score += 0.15
    
    # Season match (weight: 10%)
    if query_attrs['season']:
        max_score += 0.10
        if str(catalog_row['Season']) == query_attrs['season']:
            score += 0.10
    
    # Normalize by max possible score
    if max_score > 0:
        return score / max_score
    return 0

In [37]:
# Main matching function that returns a list of tuples in the form of SKU, Product_Name, confidence_score, match_reason
def match_product(query_text, query_attrs, catalog_df, catalog_vecs, top_k=3):
    # Get text similarity scores
    query_clean = clean_description(query_text)
    query_vec = tfidf.transform([query_clean])
    text_similarities = cosine_similarity(query_vec, catalog_vecs)[0]
    
    # Calculate attribute scores for each catalog item
    attr_scores = []
    for idx, row in catalog_df.iterrows():
        attr_score = calculate_attribute_score(query_attrs, row)
        attr_scores.append(attr_score)
    
    attr_scores = np.array(attr_scores)
    
    # Combine scores (60% text similarity, 40% attribute matching)
    final_scores = (0.6 * text_similarities) + (0.4 * attr_scores)
    
    # Get top K matches
    top_indices = np.argsort(final_scores)[-top_k:][::-1]
    
    results = []
    for idx in top_indices:
        sku = catalog_df.iloc[idx]['SKU']
        product_name = catalog_df.iloc[idx]['Product_Name']
        confidence = final_scores[idx] * 100  # Convert to percentage
        
        # Build match reason
        reasons = []
        if query_attrs['brand'] and catalog_df.iloc[idx]['brand_clean'] == query_attrs['brand'].lower():
            reasons.append(f"Brand: {query_attrs['brand']}")
        if query_attrs['color'] and catalog_df.iloc[idx]['color_clean'] == query_attrs['color'].lower():
            reasons.append(f"Color: {query_attrs['color']}")
        if query_attrs['size'] and catalog_df.iloc[idx]['size_clean'] == query_attrs['size']:
            reasons.append(f"Size: {query_attrs['size']}")
        if query_attrs['subcategory'] and catalog_df.iloc[idx]['subcategory_clean'] == query_attrs['subcategory'].lower():
            reasons.append(f"Type: {query_attrs['subcategory']}")
        
        match_reason = " | ".join(reasons) if reasons else "Text similarity match"
        
        results.append({
            'SKU': sku,
            'Product_Name': product_name,
            'Brand': catalog_df.iloc[idx]['Brand'],
            'Color': catalog_df.iloc[idx]['Color'],
            'Size': catalog_df.iloc[idx]['Size'],
            'Confidence': round(confidence, 2),
            'Match_Reason': match_reason,
            'Text_Similarity': round(text_similarities[idx] * 100, 2),
            'Attribute_Score': round(attr_scores[idx] * 100, 2)
        })
    
    return results

In [38]:
# Test the matching algorithm
test_query = "Looking for naavy Santals by lite, size L please."
test_attrs = extract_all_attributes(test_query, catalog_clean)
test_results = match_product(test_query, test_attrs, catalog_clean, catalog_vectors, top_k=3)

In [39]:
print(f"TEST QUERY: {test_query}")
print(f"Extracted Attributes: {test_attrs}")

TEST QUERY: Looking for naavy Santals by lite, size L please.
Extracted Attributes: {'brand': None, 'color': 'Navy', 'size': 'L', 'subcategory': None, 'season': None}


In [40]:
for i, result in enumerate(test_results, 1):
    print(f"\nMatch #{i}:")
    print(f"  SKU: {result['SKU']}")
    print(f"  Product: {result['Product_Name']}")
    print(f"  Brand: {result['Brand']} | Color: {result['Color']} | Size: {result['Size']}")
    print(f"  Confidence: {result['Confidence']}%")
    print(f"  Reason: {result['Match_Reason']}")


Match #1:
  SKU: SKU1000752
  Product: Alpine Sandals
  Brand: Alpine | Color: Navy | Size: L
  Confidence: 72.53%
  Reason: Color: Navy | Size: L

Match #2:
  SKU: SKU1000697
  Product: Elite Sandals
  Brand: Elite | Color: Navy | Size: L
  Confidence: 65.81%
  Reason: Color: Navy | Size: L

Match #3:
  SKU: SKU1000647
  Product: Classic Sandals
  Brand: Classic | Color: Navy | Size: S
  Confidence: 54.61%
  Reason: Color: Navy


<b> The final step to this notebook is to process all descriptions and create the suitable output.</b>

In [41]:
all_matches = []

for idx, row in descriptions_clean.iterrows():
    desc_id = row['Description_ID']
    desc_text = row['Unstructured_Description']
    desc_clean = row['description_clean']
    attrs = row['extracted_attrs']
    # Get matches
    matches = match_product(desc_text, attrs, catalog_clean, catalog_vectors, top_k=3)
    
    for rank, match in enumerate(matches, 1):
        all_matches.append({
            'Description_ID': desc_id,
            'Original_Description': desc_text,
            'Match_Rank': rank,
            'SKU': match['SKU'],
            'Product_Name': match['Product_Name'],
            'Brand': match['Brand'],
            'Color': match['Color'],
            'Size': match['Size'],
            'Confidence': match['Confidence'],
            'Match_Reason': match['Match_Reason'],
            'Text_Similarity': match['Text_Similarity'],
            'Attribute_Score': match['Attribute_Score'],
            'Extracted_Brand': attrs['brand'],
            'Extracted_Color': attrs['color'],
            'Extracted_Size': attrs['size'],
            'Extracted_Subcategory': attrs['subcategory'],
            'Extracted_Season': attrs['season']
        })

In [42]:
# Create DataFrame
matches_df = pd.DataFrame(all_matches)

# Save to CSV
matches_df.to_csv('matching_results.csv', index=False)
print(f"\nMatching complete! Results saved to 'matching_results.csv'")
print(f"Total matches generated: {len(matches_df)}")


Matching complete! Results saved to 'matching_results.csv'
Total matches generated: 750
